## 代码复杂度的衡量

In [1]:
from radon.complexity import cc_visit
from radon.metrics import h_visit
from radon.metrics import mi_visit
from cognitive_complexity.api import get_cognitive_complexity

### 行复杂度

In [17]:
def count_physical_loc(code_string):
    # 将输入的代码字符串按换行符分割成多行
    lines = code_string.split("\n")
    # 过滤掉空行（即只包含空白字符的行），只保留非空行
    non_empty_lines = [line for line in lines if line.strip() != ""]
    # 返回非空行的数量，即物理行数
    return len(non_empty_lines)

### 平均圈复杂度

圈复杂度是衡量代码逻辑复杂度的一个指标，通常用来衡量代码的可维护性和潜在的错误风险。

总圈复杂度（Total Cyclomatic Complexity）是通过累加代码中所有函数、方法、类或模块的圈复杂度值来计算的。每个代码块（如函数或方法）的圈复杂度表示其控制流图中独立路径的数量，这些路径包括条件判断、循环、异常处理等结构。

圈复杂度的计算通常基于控制流图（CFG），每当有分支或循环时，复杂度就会增加。圈复杂度的公式为：

$$
M = E - N + 2P
$$

其中：
- \( M \): 圈复杂度
- \( E \): 控制流图中的边数（Edges）
- \( N \): 控制流图中的节点数（Nodes）
- \( P \): 控制流图的连通部分数量（通常为 1）

对于大多数代码块，圈复杂度的增加规则如下：
- 每个函数默认的圈复杂度是 **1**。
- 每遇到一个 **if**、**else if**、**for**、**while**、**case**、**catch** 或 **&&**、**||** 操作符时，圈复杂度加 **1**。

In [18]:
def calculate_cyclomatic_complexity(code):
    # 分析输入代码，得到代码块列表
    blocks = cc_visit(code)
    # 循环打印每个代码块的名称和复杂度
    for block in blocks:
        print(f"{block.name}: {block.complexity}")
    # 计算所有代码块的总圈复杂度
    total_complexity = sum(block.complexity for block in blocks)
    # 如果有代码块，计算平均圈复杂度，否则返回0
    average_complexity = total_complexity / len(blocks) if blocks else 0
    # 打印平均圈复杂度
    print(f"Average Cyclomatic Complexity: {average_complexity}")
    # 返回平均圈复杂度
    return average_complexity

In [19]:
example_code = """
def example_function(x):
    if x > 0:
        print("Positive")
    elif x < 0:
        print("Negative")
    else:
        print("Zero")

    for i in range(10):
        if i % 2 == 0:
            print(f"{i} is even")
        else:
            print(f"{i} is odd")

def calculate_cyclomatic_complexity(code):
    blocks = cc_visit(code)
    for block in blocks:
        print(f"{block.name}: {block.complexity}")
    total_complexity = sum(block.complexity for block in blocks)
    average_complexity = total_complexity / len(blocks) if blocks else 0
    print(f"Average Cyclomatic Complexity: {average_complexity}")
    return average_complexity            
"""
average_complexity = calculate_cyclomatic_complexity(example_code)

example_function: 5
calculate_cyclomatic_complexity: 4
Average Cyclomatic Complexity: 4.5


### Halstead 复杂度

Halstead 复杂度是软件度量中的一种，它通过分析代码中的操作符和操作数来度量代码的复杂性。Halstead 复杂度的几个主要度量指标包括：
- `n1`: 操作符的总数量
- `n2`: 操作数的总数量
- `N1`: 操作符的不同种类（独立的操作符）
- `N2`: 操作数的不同种类（独立的操作数）
- 词汇表大小（vocabulary）: 代码中唯一的操作符和操作数的总数，即 `N = n1 + n2`。
- 程序长度（length）: 操作符和操作数的总数量，即 `N1 + N2`。
- 其他度量值还包括体积（volume）、难度（difficulty）和努力（effort）。

In [20]:
def calculate_halstead_complexity(code):
    # 使用 h_visit 函数分析代码，得到 Halstead 复杂度的结果
    results = h_visit(code)
    
    # 返回 Halstead 复杂度中的词汇表大小
    return results[0].vocabulary

In [21]:
example_code = """
def example(x, y):
    return x + y          
"""

calculate_halstead_complexity(example_code)

3

### 可维护性指数（Maintainability Index, MI）

可维护性指数是用于衡量代码质量和可维护性的一种标准度量。它综合考虑了代码的复杂性、代码行数、注释密度等多个因素，给出一个分数。分数越高，表示代码越容易维护。

In [22]:
def calculate_mi(code_string):
    # 使用 mi_visit 函数分析代码并计算可维护性指数
    mi_score = mi_visit(code_string, True)
    
    # 通过 100 减去 mi_score 的方式得到最终的可维护性指数
    return 100 - mi_score

In [23]:
code_string = """
def example_function(x):
    if x > 0:
        print("Positive")
    else:
        print("Non-positive")
"""
calculate_mi(code_string)

20.2576572942328

### 认知复杂度（Cognitive Complexity）

通过解析代码并过滤掉一些不影响复杂度的语法节点，然后调用 `get_cognitive_complexity` 函数来计算认知复杂度。认知复杂度是一种用于衡量代码理解难度的度量，考虑了嵌套深度和代码中的控制流

In [24]:
import ast

def calculate_cognitive_complexity(code):
    # 解析代码字符串为 AST（抽象语法树）
    parsed_code = ast.parse(code)
    try:
        # 过滤掉特定类型的语法节点，不计入复杂度
        new_body = [
            node
            for node in parsed_code.body
            if not isinstance(
                node,
                (
                    ast.Import,  # 导入语句
                    ast.ImportFrom,  # 从模块导入语句
                    ast.Assign,  # 赋值语句
                    ast.Expr,  # 表达式语句
                    ast.For,  # for 循环语句
                    ast.AugAssign,  # 增量赋值语句 (+=, -= 等)
                ),
            )
        ]
        # 如果没有剩余的代码块，返回一个空函数定义
        if not new_body:
            funcdef = ast.parse("")
        # 否则，使用第一个剩下的代码块
        else:
            funcdef = new_body[0]

    except Exception as e:
        # 如果发生异常，打印错误信息，并回退使用原始代码
        print(e)
        print("Using original code.")
        # 如果解析后的代码为空，抛出错误
        if not parsed_code.body:
            raise ValueError("The code provided is empty or invalid.")
        # 否则，使用解析代码的第一个部分
        funcdef = parsed_code.body[0]

    # 计算过滤后的代码块的认知复杂度
    cc_score = get_cognitive_complexity(funcdef)
    # 返回计算出的认知复杂度分数
    return cc_score

In [25]:
code_string = """
def process_numbers(numbers):
    total = 0
    for num in numbers:
        if num > 0:
            if num % 2 == 0:
                total += num
            else:
                total += num * 2
        else:
            if num == 0:
                total += 1
            else:
                total -= num
    return total
"""
calculate_cognitive_complexity(code_string)

8